In [6]:
#!pip install langchain
#!pip install unstructured[pdf]
#!pip install --upgrade openai
#!pip install tiktoken

Restart the kernel.

In [1]:
import os
import openai
from dotenv import load_dotenv

from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import AzureSearch

from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

In [2]:
# Load environment variables
load_dotenv()

True

In [6]:
# Configure OpenAI API
openai.api_type = "azure"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_version = os.getenv('OPENAI_API_VERSION')

# Initialize gpt-35-turbo and our embedding model
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo")
embeddings = OpenAIEmbeddings(deployment_id="text-embedding-ada-002", chunk_size=1)

# Connect to Azure Cognitive Search
acs = AzureSearch(azure_search_endpoint=os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT'),
                 azure_search_key=os.getenv('AZURE_SEARCH_ADMIN_KEY'),
                 index_name="openai-learning-days-mb-index",
                 embedding_function=embeddings.embed_query)

C:\Users\linapalk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain\embeddings\openai.py:214: UserWarning: WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.
  warnings.warn(


In [4]:
# Load PDF files from "data" folder. This may take few minutes. Recommended to start with 2-3 PDF files.
directory = "./data"

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
print(len(documents))

2


In [5]:
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

2190


In [7]:
# Add documents to Azure Search
acs.add_documents(documents=docs)

['...',
 ...]